In [ ]:
  !pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 31.4 MB/s 


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import csv
import pandas as pd 
import numpy as np
import datetime 
import time 
import numpy as np 
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns 
from matplotlib import rcParams   
import math
import pickle
import os
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
import networkx as nx
import pdb
import pickle
from tqdm.notebook import tqdm
import os
import random
from sklearn.model_selection import train_test_split
from pandas import read_hdf
from sklearn.metrics import f1_score

# Algo imports
import tensorflow as tf
from tensorflow import keras
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    ClusterNodeGenerator,
)
from stellargraph import StellarGraph
from stellargraph.layer import GCN, DeepGraphInfomax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import Model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_path = "gdrive/My Drive/Twitch Link Prediction/ENGB/"

In [ ]:
df_train = pd.read_csv(data_path+"engb_pos_train_91.csv", index_col=0)
print(df_train.shape)
print(df_train.head())

(31851, 3)
        src  dest  link
6694   3609  2481     1
26453  3455  6007     1
13586  1598  2955     1
10649  1165  6920     1
4589    437  4983     1


In [ ]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.DiGraph())
degrees = nx.degree(train_graph)
degrees = dict(degrees)

features = {}
for key, val in degrees.items():
  features[key] = [val]

pr = nx.pagerank(train_graph, alpha=0.85)
pr = dict(pr)
for key, val in pr.items():
  features[key].append(val)

katz = nx.katz_centrality(train_graph,alpha=0.005,beta=1)
katz = dict(katz)
for key, val in katz.items():
  features[key].append(val)

nx.set_node_attributes(train_graph, features, "features")

In [ ]:
G = StellarGraph.from_networkx(train_graph, node_features="features")

In [ ]:
print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 7126, Edges: 31851

 Node types:
  default: [7126]
    Features: float32 vector, length 3
    Edge types: default-default->default

 Edge types:
    default-default->default: [31851]
        Weights: all 1 (default)
        Features: none


In [ ]:
fullbatch_generator = FullBatchNodeGenerator(G, sparse=True)
gcn_model = GCN(layer_sizes=[64], activations=['relu'], generator=fullbatch_generator)

corrupted_generator = CorruptedGenerator(fullbatch_generator)
gen = corrupted_generator.flow(G.nodes())

Using GCN (local pooling) filters...


In [ ]:
infomax = DeepGraphInfomax(gcn_model, corrupted_generator)
x_in, x_out = infomax.in_out_tensors()

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=tf.nn.sigmoid_cross_entropy_with_logits, optimizer=Adam(lr=1e-3))

In [ ]:
batch_size=100
epochs=50
es = EarlyStopping(monitor="loss", min_delta=0, patience=4, restore_best_weights=True,verbose=1)

In [ ]:
history = model.fit(gen, epochs=epochs, verbose=1, callbacks=[es])

Epoch 1/50
1/1 [==============================] - 12s 12s/step - loss: 10.4562
Epoch 2/50
1/1 [==============================] - 0s 153ms/step - loss: 9.2572
Epoch 3/50
1/1 [==============================] - 0s 89ms/step - loss: 8.0686
Epoch 4/50
1/1 [==============================] - 0s 111ms/step - loss: 6.8911
Epoch 5/50
1/1 [==============================] - 0s 123ms/step - loss: 5.7376
Epoch 6/50
1/1 [==============================] - 0s 76ms/step - loss: 4.5901
Epoch 7/50
1/1 [==============================] - 0s 73ms/step - loss: 3.4690
Epoch 8/50
1/1 [==============================] - 0s 92ms/step - loss: 2.3832
Epoch 9/50
1/1 [==============================] - 0s 138ms/step - loss: 1.3716
Epoch 10/50
1/1 [==============================] - 0s 68ms/step - loss: 0.6925
Epoch 11/50
1/1 [==============================] - 0s 106ms/step - loss: 0.8197
Epoch 12/50
1/1 [==============================] - 0s 80ms/step - loss: 1.1496
Epoch 13/50
1/1 [==============================] - 0s 7

In [ ]:
x_emb_in, x_emb_out = gcn_model.in_out_tensors()

x_out = tf.squeeze(x_emb_out, axis=0)
emb_model = Model(inputs=x_emb_in, outputs=x_out)

In [ ]:
train_gen = fullbatch_generator.flow(G.nodes())
emb = emb_model.predict(train_gen)

In [ ]:
emb_dict = {}
for i, node in enumerate(list(G.nodes())):
  emb_dict[node] = np.array(emb[i].reshape(1,64))

In [ ]:
combined = pd.read_csv(data_path+"/engb_train_91.csv", index_col = 0)
combined.head()

,src,dest,link
3746,3387,512,0
32060,3625,2015,0
29151,4080,4876,1
8290,2633,6791,0
29003,4044,4799,1


In [ ]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  train_emb.append(comb_emb[0])
train_emb = np.array(train_emb)

0it [00:00, ?it/s]

In [ ]:
train_y = combined['link']
train_y.shape

(63351,)

In [ ]:
combined_test = pd.read_csv(data_path+"/engb_test_91.csv", index_col = 0)
combined_test.head()

,src,dest,link
17669,2061,2143,1
17352,2021,2450,1
26652,3499,4913,1
23292,1992,3967,0
4752,452,4671,1


In [ ]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_test.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  test_emb.append(comb_emb[0])

print(type(test_emb))
test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_test['link'] 

0it [00:00, ?it/s]

<class 'list'>
(6973, 64)


In [ ]:
#train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.5, random_state = 42)

In [ ]:
#reading
df_final_train = read_hdf(data_path + 'engb_fea_sample/engb_storage_sample_stage4.h5', 'train_df',mode='r')
df_final_test = read_hdf(data_path + 'engb_fea_sample/engb_storage_sample_stage4.h5', 'test_df',mode='r')

In [ ]:
y_train = df_final_train.link
y_test = df_final_test.link

In [ ]:
df_final_train.drop(['src', 'dest','link'],axis=1,inplace=True)
df_final_test.drop(['src', 'dest','link'],axis=1,inplace=True)

In [ ]:
# GCN only 
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.7693721695827275 test Score 0.7197621655452981


In [ ]:
# GCN only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.7750109217999126 test Score 0.7188881869867341


In [ ]:
# GCN only
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.7612052416325849 test Score 0.7229329173166926


In [ ]:
# Heuristic only
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9727737078054952 test Score 0.74853195164076


In [ ]:
# Heuristic only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9666640764628177 test Score 0.7794803464357095


In [ ]:
# Heuristic only
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9737882742774974 test Score 0.7443791075752335


In [ ]:
df_final_train = np.hstack((df_final_train, train_emb))
print(df_final_train.shape)
df_final_test = np.hstack((df_final_test, test_emb))
print(df_final_test.shape)

(63351, 120)
(6973, 120)


In [ ]:
print(type(train_y), type(y_train))
for i in range(len(y_train.values)):
  if train_y.values[i] != y_train.values[i]: print(i)

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [ ]:
# Heuristic + GCN
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9737125720444216 test Score 0.7479253112033195


In [ ]:
# Heuristic + GCN
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9640528443322439 test Score 0.7860304968027546


In [ ]:
# Heuristic + GCN
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9739144017494533 test Score 0.7448538315170387
